<a href="https://colab.research.google.com/github/arthurpham/google_colab/blob/main/TARF_MC_Performance_TQF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install --upgrade tensorflow --user
!pip install tf-quant-finance
!pip install QuantLib-Python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
# reduce number of threads
os.environ['TF_NUM_INTEROP_THREADS'] = '1'
os.environ['TF_NUM_INTRAOP_THREADS'] = '1'

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import tf_quant_finance as tff 
import tensorflow as tf
import functools
import pandas as pd
import time
import QuantLib as ql

In [3]:
!nvidia-smi

Wed May 25 15:17:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
spot = 18.0
strike = 20.0
K_lower = 15.0
K_upper = 20.0
K_knockout = 30.0
tarf_target = 5.0
step_up_ratio = 2.0

r = 0.0
volatility = 0.5

In [5]:
#@title Set up parameters

dtype = tf.float64 #@param
num_samples = 200000 #@param
num_timesteps = 53 #@param

# expiries =tf.constant( [0.0, 0.5, 1.0], dtype=dtype) # This can be a rank 1 Tensor
dt = 1. / num_timesteps
# times = [1.0]
times = tf.linspace(tf.constant(0.0, dtype=dtype), tf.constant(1.0, dtype=dtype), num_timesteps)
rate = tf.constant(r, dtype=dtype)
sigma = tf.constant(volatility, dtype=dtype)
spot = tf.constant(spot, dtype=dtype)
strikes = tf.constant(strike, dtype=dtype)

def set_up_pricer(times, watch_params=False):
    """Set up European option pricing function under Black-Scholes model.
    
    Args:
        expiries: List of expiries at which to to sample the trajectories.
        watch_params: A Python bool. When `True`, gradients of the price function wrt the inputs
          are computed more efficiently. 
    Returns:
     A callable that accepts a rank 1 tensor of strikes, and scalar values for 
     the spots and  volatility values. The callable outputs prices of
     the European call options on the grid `expiries x strikes`.
    """
    def price_eu_options(strikes, spot, sigma):
        # Define drift and volatility functions. 
        def drift_fn(t, x):
          del t, x
          return rate - 0.5 * sigma**2
        def vol_fn(t, x):
          del t, x
          return tf.reshape(sigma, [1, 1])
        # Use GenericItoProcess class to set up the Ito process
        process = tff.models.GenericItoProcess(
            dim=1,
            drift_fn=drift_fn,
            volatility_fn=vol_fn,
            dtype=dtype)
        log_spot = tf.math.log(tf.reduce_mean(spot))
        if watch_params:
            watch_params_list = [sigma]
        else:
            watch_params_list = None
        paths = process.sample_paths(
            times=times, num_samples=num_samples,
            initial_state=log_spot, 
            watch_params=watch_params_list,
            # Select a random number generator
            random_type=tff.math.random.RandomType.SOBOL, #PSEUDO_ANTITHETIC
            time_step=dt)
        
        @tf.function
        def tarf_payoff(element):
#             tf.print(element, summarize=-1)
            total = tf.constant(0.0, dtype=tf.float64)
            discounted_payoff = tf.constant(0.0, dtype=tf.float64)
            df = tf.constant(1.0, dtype=tf.float64)
            is_active = True
            for cur_spot in element:
                if is_active:
                    cashflow = tf.constant(0.0, dtype=tf.float64)
                    add_cashflow = False
                    if K_knockout <= cur_spot:
                        # early termination
                        is_active = False
                    if K_upper <= cur_spot: # cur_spot < K_knockout
                        cashflow = cur_spot - strike
                        add_cashflow = True
                    if cur_spot < K_lower:
                        cashflow = step_up_ratio*(cur_spot - strike)
                        add_cashflow = True

                    if add_cashflow:
                        if total + cashflow >= tarf_target:
                            cashflow = tarf_target - total
                            total += cashflow
                            discounted_payoff += df*cashflow
                            is_active = False
                        else:
                            total += cashflow
                            discounted_payoff += df*cashflow
                     
            return discounted_payoff

        reshaped_paths = tf.reshape(tf.math.exp(paths), [num_samples, num_timesteps])
        payoffs = tf.vectorized_map(tarf_payoff, reshaped_paths)
        prices = tf.reduce_mean(payoffs)

        return prices
    return price_eu_options    

price_eu_options = tf.function(set_up_pricer(times, watch_params=True),
                               input_signature=[
                                                tf.TensorSpec([], dtype=tf.float64),
                                                tf.TensorSpec([], dtype=tf.float64),
                                                tf.TensorSpec([], dtype=tf.float64)
                               ])


In [6]:
def set_up_pricer_xla(times, watch_params=False):
    """Set up European option pricing function under Black-Scholes model.
    
    Args:
        expiries: List of expiries at which to to sample the trajectories.
        watch_params: A Python bool. When `True`, gradients of the price function wrt the inputs
          are computed more efficiently. 
    Returns:
     A callable that accepts a rank 1 tensor of strikes, and scalar values for 
     the spots and  volatility values. The callable outputs prices of
     the European call options on the grid `expiries x strikes`.
    """
    def price_eu_options(strikes, spot, sigma):
        # Define drift and volatility functions. 
        def drift_fn(t, x):
          del t, x
          return rate - 0.5 * sigma**2
        def vol_fn(t, x):
          del t, x
          return tf.reshape(sigma, [1, 1])
        # Use GenericItoProcess class to set up the Ito process
        process = tff.models.GenericItoProcess(
            dim=1,
            drift_fn=drift_fn,
            volatility_fn=vol_fn,
            dtype=dtype)
        log_spot = tf.math.log(tf.reduce_mean(spot))
        if watch_params:
            watch_params_list = [sigma]
        else:
            watch_params_list = None
        paths = process.sample_paths(
            times=times, num_samples=num_samples,
            initial_state=log_spot, 
            watch_params=watch_params_list,
            # Select a random number generator
            random_type=tff.math.random.RandomType.SOBOL, #PSEUDO_ANTITHETIC
            time_step=dt)
        
        @tf.function
        def tarf_payoff(paths):
            # Shape [num_timesteps, num_samples]
            paths = tf.transpose(paths)
            cur_spot = paths[0]
            total = tf.zeros_like(cur_spot)
            discounted_payoff = tf.zeros_like(cur_spot)
            df = tf.constant(1.0, dtype=tf.float64)
            is_active = tf.ones([num_samples], dtype=tf.bool)
            i = tf.constant(0, dtype=tf.int32)
            # Explicitly define the while_loop 
            def cond(i, is_active, total, discounted_payoff):
                return i < num_timesteps

            def body(i, is_active, total, discounted_payoff):
                # Here Tensors are of shape `[num_samples]`
                cur_spot = paths[i]

                cashflow = tf.zeros_like(cur_spot)
                new_is_active = K_knockout > cur_spot
                add_cashflow = tf.where(tf.logical_or(K_upper <= cur_spot, cur_spot < K_lower),
                    True, 
                    False)

                new_cashflow = tf.where(K_upper <= cur_spot,
                    cur_spot - strike,
                    cashflow
                )
                new_cashflow = tf.where(cur_spot < K_lower,
                                    step_up_ratio*(cur_spot - strike),
                                    new_cashflow)
                new_is_active = tf.where(add_cashflow,
                    tf.where(total + new_cashflow >= tarf_target,
                            False, new_is_active),
                    new_is_active)

                new_cashflow = tf.where(add_cashflow,
                    tf.where(total + new_cashflow >= tarf_target,
                            tarf_target - total, new_cashflow),
                    new_cashflow
                    )

                new_total = tf.where(add_cashflow,
                    total + new_cashflow,
                    total
                    )
                new_discounted_payoff = tf.where(add_cashflow,
                    discounted_payoff + df * new_cashflow,
                    discounted_payoff)
                # Update values only if active
                new_cashflow = tf.where(is_active, 
                                        new_cashflow,
                                        cashflow)
                new_total = tf.where(is_active, new_total, total)
                new_discounted_payoff = tf.where(is_active, 
                                        new_discounted_payoff,
                                        discounted_payoff)
                new_is_active = tf.where(is_active, new_is_active, is_active)

                return (i + 1, new_is_active, new_total, new_discounted_payoff)
                
            _, is_active, total, discounted_payoff = tf.while_loop(
                cond, body, (i, is_active, total, discounted_payoff)
            )
            return discounted_payoff

        reshaped_paths = tf.reshape(tf.math.exp(paths), [num_samples, num_timesteps])
        payoffs = tarf_payoff(reshaped_paths)
        prices = tf.reduce_mean(payoffs)

        return prices
    return price_eu_options

price_eu_options_xla = tf.function(set_up_pricer_xla(times, watch_params=True),
                               input_signature=[
                                                tf.TensorSpec([], dtype=tf.float64),
                                                tf.TensorSpec([], dtype=tf.float64),
                                                tf.TensorSpec([], dtype=tf.float64)
                               ], jit_compile=True)    

In [7]:
# device = "/gpu:0"
# with tf.device(device):
#     tarf_price = price_eu_options(strikes, spot, sigma)
#     print('price', tarf_price)

In [8]:
# device = "/gpu:0"
# with tf.device(device):
#     tarf_price = price_eu_options_xla(strikes, spot, sigma)
#     print('price', tarf_price)

In [9]:
devices = ['gpu', 'cpu']

for device in devices:
    with tf.device('/{}:0'.format(device)):
        t = time.time()
        tarf_price = price_eu_options(strikes, spot, sigma)
        time_tqf0 = time.time() - t

        t = time.time()
        tarf_price = price_eu_options(strikes, spot, sigma)
        time_tqf = time.time() - t

        print('------------------------')
        print('TQF {} TARF'.format(device))
        print('wall time + tracing: ', time_tqf0)
        print('options per second + tracing: ', 1.0/time_tqf0)
        print('wall time: ', time_tqf)
        print('options per second: ', 1.0/time_tqf)
        print('------------------------')
        print('price', tarf_price)

------------------------
TQF gpu TARF
wall time + tracing:  23.77046799659729
options per second + tracing:  0.04206900765029736
wall time:  3.145033121109009
options per second:  0.31796167528034736
------------------------
price tf.Tensor(-246.927357070608, shape=(), dtype=float64)
------------------------
TQF cpu TARF
wall time + tracing:  11.315168857574463
options per second + tracing:  0.08837694006931165
wall time:  8.648904800415039
options per second:  0.11562157557243694
------------------------
price tf.Tensor(-246.927357070608, shape=(), dtype=float64)


In [10]:
devices = ['gpu', 'cpu']

for device in devices:
    with tf.device('/{}:0'.format(device)):
        t = time.time()
        tarf_price = price_eu_options_xla(strikes, spot, sigma)
        time_tqf0 = time.time() - t

        t = time.time()
        tarf_price = price_eu_options_xla(strikes, spot, sigma)
        time_tqf = time.time() - t

        print('------------------------')
        print('TQF {} TARF XLA'.format(device))
        print('wall time + tracing: ', time_tqf0)
        print('options per second + tracing: ', 1.0/time_tqf0)
        print('wall time: ', time_tqf)
        print('options per second: ', 1.0/time_tqf)
        print('------------------------')
        print('price', tarf_price)

------------------------
TQF gpu TARF XLA
wall time + tracing:  3.409421920776367
options per second + tracing:  0.2933048543819674
wall time:  0.4923841953277588
options per second:  2.0309343993755595
------------------------
price tf.Tensor(-246.927357070608, shape=(), dtype=float64)
------------------------
TQF cpu TARF XLA
wall time + tracing:  12.18169093132019
options per second + tracing:  0.08209040975000545
wall time:  11.571342945098877
options per second:  0.08642039258058262
------------------------
price tf.Tensor(-246.927357070608, shape=(), dtype=float64)


In [11]:
@tf.function(jit_compile=False,
             input_signature=[tf.TensorSpec([], dtype=tf.float64),
                            tf.TensorSpec([], dtype=tf.float64),
                            tf.TensorSpec([], dtype=tf.float64)
                               ])
def delta_fn(strikes, spot, sigma):
    fn = lambda spot: price_eu_options(strikes, spot, sigma)
    return tff.math.gradients(fn, spot, use_gradient_tape=True)

@tf.function(jit_compile=True,
             input_signature=[tf.TensorSpec([], dtype=tf.float64),
                            tf.TensorSpec([], dtype=tf.float64),
                            tf.TensorSpec([], dtype=tf.float64)
                               ])
def delta_fn_xla(strikes, spot, sigma):
    fn = lambda spot: price_eu_options_xla(strikes, spot, sigma)
    return tff.math.gradients(fn, spot, use_gradient_tape=True)

In [12]:
@tf.function(jit_compile=False,
             input_signature=[tf.TensorSpec([], dtype=tf.float64),
                            tf.TensorSpec([], dtype=tf.float64),
                            tf.TensorSpec([], dtype=tf.float64)
                               ])
def vega_fn(strikes, spot, sigma):
    fn = lambda sigma: price_eu_options(strikes, spot, sigma)
    return tff.math.gradients(fn, sigma, use_gradient_tape=True)

In [13]:
devices = ['gpu', 'cpu']

for device in devices:
    with tf.device('/{}:0'.format(device)):
        t = time.time()
        tarf_price = price_eu_options_xla(strikes, spot, sigma)
        tarf_delta = delta_fn(strikes, spot, sigma)
        tarf_vega = vega_fn(strikes, spot, sigma)
        time_tqf0 = time.time() - t

        t = time.time()
        tarf_price = price_eu_options_xla(strikes, spot, sigma)
        tarf_delta = delta_fn(strikes, spot, sigma)
        tarf_vega = vega_fn(strikes, spot, sigma)
        time_tqf = time.time() - t

        print('------------------------')
        print('TQF {} TARF price+delta+vega'.format(device))
        print('wall time + tracing: ', time_tqf0)
        print('options per second + tracing: ', 1.0/time_tqf0)
        print('wall time: ', time_tqf)
        print('options per second: ', 1.0/time_tqf)
        print('------------------------')
        print('price', tarf_price)
        print('delta', tarf_delta)
        print('vega', tarf_vega)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradients/while/cond/pfor/GatherV2_grad/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradients/while/cond/pfor/GatherV2_grad/Reshape:0", shape=(None,), dtype=float64), dense_shape=Tensor("gradients/while/cond/pfor/GatherV2_grad/Cast:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradients/while_grad/Identity_4:0", shape=(None,), dtype=int32), values=Tensor("gradients/while_grad/Identity_3:0", shape=(None,), dtype=float64), dense_shape=Tensor("gradients/while_grad/Identity_5:0", shape=(1,), dtype=int32))) to a dense Tensor of 

------------------------
TQF gpu TARF price+delta+vega
wall time + tracing:  23.602126359939575
options per second + tracing:  0.04236906390338299
wall time:  14.480068922042847
options per second:  0.06906044476609577
------------------------
price tf.Tensor(-246.927357070608, shape=(), dtype=float64)
delta tf.Tensor(21.37942877385511, shape=(), dtype=float64)
vega tf.Tensor(-331.98229464223175, shape=(), dtype=float64)
------------------------
TQF cpu TARF price+delta+vega
wall time + tracing:  53.9759304523468
options per second + tracing:  0.018526776502405275
wall time:  42.03431582450867
options per second:  0.023790086275579075
------------------------
price tf.Tensor(-246.927357070608, shape=(), dtype=float64)
delta tf.Tensor(21.37942877385511, shape=(), dtype=float64)
vega tf.Tensor(-331.9822946422317, shape=(), dtype=float64)


In [14]:
devices = ['gpu', 'cpu']

for device in devices:
    with tf.device('/{}:0'.format(device)):
        t = time.time()
        tarf_delta = delta_fn_xla(strikes, spot, sigma)
        time_tqf0 = time.time() - t

        t = time.time()
        tarf_delta = delta_fn_xla(strikes, spot, sigma)
        time_tqf = time.time() - t

        print('------------------------')
        print('TQF {} TARF'.format(device))
        print('wall time + tracing: ', time_tqf0)
        print('options per second + tracing: ', 1.0/time_tqf0)
        print('wall time: ', time_tqf)
        print('options per second: ', 1.0/time_tqf)
        print('------------------------')
        print('delta', tarf_delta)

InvalidArgumentError: ignored